<a href="https://colab.research.google.com/github/ttury/Deep-Learning-For-Natural-Language-Processing/blob/master/Cosine_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from numpy import dot # 이거 안해주면 앞에 np. 붙여야되서 귀찮음
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
  return dot(A, B) / (norm(A) * norm(B))

In [3]:
import pandas as pd
from math import log

docs = [
        '저는 사과 좋아요',
        '저는 바나나 좋아요',
        '저는 바나나 좋아요 저는 바나나 좋아요'
]
vocab = set()
for doc in docs:
  vocab.update(doc.split())
vocab = list(vocab)
vocab.sort()

In [4]:
num_of_docs = len(docs)

def term_frequency(term, doc):
  return doc.count(term)

def doc_frequency(term):
  df = 0
  for doc in docs:
    df += term in doc
  return df

def inv_doc_frequency(term):
  df = doc_frequency(term)
  return log(num_of_docs / (df + 1))

def tf_idf(term, doc):
  return term_frequency(term, doc) * inv_doc_frequency(term)

In [ ]:
result = list()
for doc in docs:
  result.append([])
  for term in vocab:
    result[-1].append(term_frequency(term, doc))

tf = pd.DataFrame(result, columns = vocab)
print(tf)

In [ ]:
doc1 = tf.loc[0].to_numpy()
doc2 = tf.loc[1].to_numpy()
doc3 = tf.loc[2].to_numpy()

print(cos_sim(doc1, doc2))
print(cos_sim(doc2, doc3))
print(cos_sim(doc3, doc1))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

data = pd.read_csv('/content/drive/MyDrive/datasets/movies_metadata.csv', low_memory = False)
data.head(2) # 데이터가 어떤 형식을 갖고 있는지 확ㅇ니

In [ ]:
data = data.head(20000)
print(data['overview'].isnull().sum())
# tf-idf 과정에서 에러를 막기 위해 null 값이 있는지 확인
data['overview'] = data['overview'].fillna('')
# null 값을 빈 값으로 대체
print(data['overview'].isnull().sum())

In [ ]:
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)
print()
# 20000개의 영화를 표현하기 위해 47487개의 단어가 사용(단어 집합의 열 개수)

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) # 코사인 유사도 구하기
# 행과 열이 모두 영화(20000 x 20000 크기 행렬), 특정 영화에 대한 나머지 영화들의 코사인 유사도 저장

indices = pd.Series(data.index, index = data['title']) # value와 index 바꿈

print(indices.head())
print()

index = indices['Father of the Bride Part II']
print(index)
# 영화의 제목을 입력하면 인덱스를 리턴

In [56]:
def get_recommendations(title, cosine_sim = cosine_sim):
  index = indices[title]
  # 영화의 제목을 이용해 인덱스 받아오기
  sim_scores = list(enumerate(cosine_sim[index]))
  # 모든 영화에서 해당 영화와의 코사인 유사도 받기, enumerate를 통해 인덱스(영화 분류)를 가진 2차원 배열 반환
  sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True) # 코사인 유사도를 기준으로 정렬
  sim_scores = sim_scores[1:11]
  # 가장 유사한 10개의 영화만 추리기(자신 제외)
  movie_indices = [sim_score[0] for sim_score in sim_scores]
  # enumerate를 통해 받아온 영화 인덱스 가져오기
  return data['title'].iloc[movie_indices]
  # index = indices[title] 과 반대 작업, data['title'][movie_indices]로 해도 됨

In [67]:
get_recommendations("Jaws")

3998              Jaws: The Revenge
1339                         Jaws 2
1340                       Jaws 3-D
16168                      The Reef
19065    Iria: Zeiram the Animation
19744                          Bait
18241                    Sharktopus
12712                Turn the River
18641    Mega Shark vs. Crocosaurus
8180                     Shark Tale
Name: title, dtype: object